# References 
1. [Detailed API Introduction](https://www.kaggle.com/sohier/detailed-api-introduction)
2. [Basic Submission Template](https://www.kaggle.com/sohier/basic-submission-template)

In [ ]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
import gresearch_crypto
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

VALIDATION_CSV = '../input/g-research-crypto-forecasting/supplemental_train.csv'
TRAIN_CSV = '/kaggle/input/g-research-crypto-forecasting/train.csv'
TEST_CSV = '../input/g-research-crypto-forecasting/example_test.csv'
ASSET_DETAILS_CSV = '/kaggle/input/g-research-crypto-forecasting/asset_details.csv'

In [ ]:
# data Load
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)
val_df = pd.read_csv(VALIDATION_CSV)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
val_df.head()

In [ ]:
## data type  
# object -> category
train_df = train_df.astype({'Asset_ID': 'category'})
test_df = test_df.astype({'Asset_ID': 'category'})
val_df = val_df.astype({'Asset_ID': 'category'})

In [ ]:
train_df.info()

In [ ]:
x_train = train_df.drop('Target', axis=1)
y_train = train_df['Target']

In [ ]:
x_valid = val_df.drop('Target', axis=1)
y_valid = val_df['Target']

In [ ]:
lgbm_model = LGBMRegressor(random_state=0)

In [ ]:
df_asset_details = pd.read_csv(ASSET_DETAILS_CSV).sort_values("Asset_ID")
df_asset_details

In [ ]:
def get_features(df):
    df_feat = df[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']].copy()
    return df_feat

In [ ]:
def get_Xy_and_model_for_asset(train_df, asset_id):
    df = train_df[train_df["Asset_ID"] == asset_id]
    
    # TODO: Try different features here!
    df_proc = get_features(df)
    df_proc['y'] = df['Target']
    df_proc = df_proc.dropna(how="any")
    
    X = df_proc.drop("y", axis=1)
    y = df_proc["y"]
    
    
    model = LGBMRegressor(random_state=0)
    model.fit(X, y)
    return X, y, model

In [ ]:
Xs = {}
ys = {}
models = {}

for asset_id, asset_name in zip(df_asset_details['Asset_ID'], df_asset_details['Asset_Name']):
    print(f"Training model for {asset_name:<16} (ID={asset_id:<2})")
    X, y, model = get_Xy_and_model_for_asset(train_df, asset_id)    
    Xs[asset_id], ys[asset_id], models[asset_id] = X, y, model

In [ ]:
x = get_features(train_df.iloc[1])
y_pred = models[0].predict([x])
y_pred[0]

In [ ]:
env = gresearch_crypto.make_env()
iter_test = env.iter_test()

for i, (df_test, df_pred) in enumerate(iter_test):
    for j , row in df_test.iterrows():
        
        model = models[row['Asset_ID']]
        x_test = get_features(row)
        y_pred = model.predict([x_test])[0]
        
        df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = y_pred
        
        
        # Print just one sample row to get a feeling of what it looks like
        if i == 0 and j == 0:
            display(x_test)

    # Display the first prediction dataframe
    if i == 0:
        display(df_pred)

    # Send submissions
    env.predict(df_pred)